In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove6b/glove.6B.200d.txt
/kaggle/input/glove6b/glove.6B.50d.txt
/kaggle/input/glove6b/glove.6B.300d.txt
/kaggle/input/glove6b/glove.6B.100d.txt
/kaggle/input/welfake/WELFake_Dataset.csv


In [2]:
import pandas as pd

df = pd.read_csv('/kaggle/input/welfake/WELFake_Dataset.csv')
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [3]:
df = df.drop('Unnamed: 0', axis = 1)
df.dropna(inplace = True)

In [4]:
df.head()

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71537 entries, 0 to 72133
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   71537 non-null  object
 1   text    71537 non-null  object
 2   label   71537 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


In [2]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [6]:
# selecting 1000 rows randomly
#df = df.sample(n = 10000, random_state = 42)
#df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 54359 to 70309
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1000 non-null   object
 1   text    1000 non-null   object
 2   label   1000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 31.2+ KB


# Pre-processing

In [4]:
#removing other than alphates and digits
import re
df['title'] = df['title'].str.replace(r"[^\w\s]", "").str.replace(r"\d", "")
df['text'] = df['text'].str.replace(r"[^\w\s]","").str.replace(r"\d", "")
df.head()

/tmp/ipykernel_28/3271493751.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title'] = df['title'].str.replace(r"[^\w\s]", "").str.replace(r"\d", "")
/tmp/ipykernel_28/3271493751.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace(r"[^\w\s]","").str.replace(r"\d", "")


,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,UNBELIEVABLE OBAMAS ATTORNEY GENERAL SAYS MOST...,Now most of the demonstrators gathered last n...,1
3,Bobby Jindal raised Hindu uses story of Christ...,A dozen politically active pastors came here f...,0
4,SATAN Russia unvelis an image of its terrifyi...,The RS Sarmat missile dubbed Satan will repla...,1
5,About Time Christian Group Sues Amazon and SPL...,All we can say on this one is it s about time ...,1


In [7]:
df.head()

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,UNBELIEVABLE OBAMAS ATTORNEY GENERAL SAYS MOST...,Now most of the demonstrators gathered last n...,1
3,Bobby Jindal raised Hindu uses story of Christ...,A dozen politically active pastors came here f...,0
4,SATAN Russia unvelis an image of its terrifyi...,The RS Sarmat missile dubbed Satan will repla...,1
5,About Time Christian Group Sues Amazon and SPL...,All we can say on this one is it s about time ...,1


In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


nltk.download('stopwords')
nltk.download('punkt')

# Function to tokenize and remove stopwords from text
def tokenize_and_remove_stopwords(text):
    # Convert the text to lowercase to ensure consistency
    text = text.lower()

    # Get the set of English stopwords from NLTK
    stop_words = set(stopwords.words('english'))

    # Tokenize the text into words
    tokens = word_tokenize(text)

    # Remove stopwords from the list of tokens
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Reconstruct the text by joining the remaining tokens
    filtered_text = " ".join(filtered_tokens)

    return filtered_text

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# Apply the function to the 'text_column' and create a new column 'filtered_text_column'
df['title'] = df['title'].apply(tokenize_and_remove_stopwords)
df['text'] = df['text'].apply(tokenize_and_remove_stopwords)
df.head()


,title,text,label
0,law enforcement high alert following threats c...,comment expected barack obama members fyf fuky...,1
2,unbelievable obamas attorney general says char...,demonstrators gathered last night exercising c...,1
3,bobby jindal raised hindu uses story christian...,dozen politically active pastors came private ...,0
4,satan russia unvelis image terrifying new supe...,rs sarmat missile dubbed satan replace ss flie...,1
5,time christian group sues amazon splc designat...,say one time someone sued southern poverty law...,1


In [10]:
df.groupby('label').describe()

title                                                                 \
      count unique                                                top freq   
label                                                                        
0       485    485  mirage return manufacturing greatness new york...    1   
1       515    512               death millions yemenis forgotten war    2   

       text                                                                 
      count unique                                                top freq  
label                                                                       
0       485    483  killing obama administration rules dismantling...    2  
1       515    505                                                       8

In [7]:
#X = df[['text', 'title']]
X = df['text']+df['title']
y = df['label']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.2, random_state = 42, stratify = y)

In [ ]:
type(X_train)

In [33]:
X_train.head()

1805     failing denounce renounce endorsement ku klux ...
55519    walt disney company ceo robert iger quit role ...
7267     seems colin kaepernick become influence youths...
46862    included among many uplifting economic numbers...
30617    former officer peter liang serve time prison f...
dtype: object

# bert cnn

In [19]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, Flatten, Dense

con1 = Conv1D(filters=32, kernel_size=3, activation='relu')(outputs['sequence_output'])
max_p1 = MaxPooling1D(pool_size=2, strides=2, padding='valid')(con1)
b = BatchNormalization()(max_p1)  # Apply BatchNormalization after MaxPooling
con2 = Conv1D(filters=32, kernel_size=3, activation='relu')(b)
max_p2 = MaxPooling1D(pool_size=2, strides=2, padding='valid')(con2)
f = Flatten()(max_p2)
d1 = Dense(128, activation='relu')(f)
output_layer = Dense(1, activation='sigmoid')(d1)

model_1 = keras.Model(inputs=[bert_input], outputs=output_layer)


In [ ]:
# Compile the model with an appropriate loss function and optimizer
model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_1.summary()

hist = model_1.fit(X_train, y_train, validation_data = (X_test, y_test),batch_size=20, epochs=20, verbose=2)
accr = model_1.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy', accr[1])


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

# bert_model (cnn + bilstm) 

In [15]:
# bert embedding
import tensorflow_text as text
import tensorflow_hub as hub

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [26]:
print(outputs)

{'encoder_outputs': [<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTenso

In [16]:
# bert embedding
import tensorflow as tf
from tensorflow.keras import layers

bert_input = tf.keras.layers.Input(shape= (), dtype = tf.string, name = 'text')
preprocessed_text = bert_preprocess(bert_input)
outputs = bert_encoder(preprocessed_text)


In [17]:
print(outputs['sequence_output'].shape)

(None, 128, 768)


In [18]:
# bert layers

num_classes = 1

conv1 = layers.Conv1D(filters = 32,kernel_size = 3, activation = 'relu')(outputs['sequence_output'])
max_pool1 = layers.MaxPooling1D(pool_size = 1, strides =2, padding = 'valid')(conv1)
l2 = layers.BatchNormalization()(max_pool1)
conv2 = layers.Conv1D(filters = 32,kernel_size = 3, activation = 'relu')(l2)
max_pool2 = layers.MaxPooling1D(pool_size = 1, strides =2, padding = 'valid')(conv2)
l3 = layers.BatchNormalization()(max_pool2)
# BiLSTM layer with 64 units
bilstm = layers.Bidirectional(layers.LSTM(units=64, activation='relu'))(l3)
dense1 = layers.Dense(units=64, activation='relu')(bilstm)
output_layer = layers.Dense(units=num_classes, activation='sigmoid')(dense1)

# Create the model
model = tf.keras.Model(inputs=[bert_input], outputs=output_layer)



In [ ]:
model.compile( 
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics =  ['binary_accuracy'],
             )
hist = model.fit(X_train,y_train,validation_data = (X_test, y_test),epochs = 20, batch_size = 20, verbose = 2)

Epoch 1/20
2862/2862 - 1041s - loss: 0.2006 - binary_accuracy: 0.9170 - val_loss: 0.1438 - val_binary_accuracy: 0.9431 - 1041s/epoch - 364ms/step
Epoch 2/20
2862/2862 - 1023s - loss: 0.1215 - binary_accuracy: 0.9528 - val_loss: 0.1443 - val_binary_accuracy: 0.9451 - 1023s/epoch - 357ms/step
Epoch 3/20
2862/2862 - 978s - loss: 0.0918 - binary_accuracy: 0.9649 - val_loss: 0.1081 - val_binary_accuracy: 0.9590 - 978s/epoch - 342ms/step
Epoch 4/20
2862/2862 - 1015s - loss: 0.0701 - binary_accuracy: 0.9733 - val_loss: 0.1072 - val_binary_accuracy: 0.9613 - 1015s/epoch - 355ms/step
Epoch 5/20
2862/2862 - 1025s - loss: 0.0541 - binary_accuracy: 0.9799 - val_loss: 0.2051 - val_binary_accuracy: 0.9434 - 1025s/epoch - 358ms/step
Epoch 6/20
2862/2862 - 978s - loss: 0.0434 - binary_accuracy: 0.9839 - val_loss: 0.1134 - val_binary_accuracy: 0.9630 - 978s/epoch - 342ms/step
Epoch 7/20
2862/2862 - 1018s - loss: 0.0361 - binary_accuracy: 0.9868 - val_loss: 0.1652 - val_binary_accuracy: 0.9540 - 1018s/e

In [ ]:
from matplotlib.pyplot import plot
history_df = pd.DataFrame(hist.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-entropy")
history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")

In [15]:
# my_model 
# bert layers
import tensorflow as tf
from tensorflow.keras import layers

text_input = tf.keras.layers.Input(shape= (), dtype = tf.string, name = 'text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

num_classes = 1


l1 = layers.BatchNormalization()(outputs['sequence_output'])
conv1 = layers.Conv1D(filters = 32,kernel_size = 3, activation = 'relu')(l1)
max_p1 = layers.MaxPooling1D(pool_size = 2, strides = 2, padding = 'valid')(conv1)
l_d1 = layers.Dropout(0.2)(max_p1)
l2 = layers.BatchNormalization()(l_d1)
conv2 = layers.Conv1D(filters = 32, kernel_size = 3, activation = 'relu')(l2)
max_p2 = layers.MaxPooling1D(pool_size = 2, strides =2, padding = 'valid')(conv2)
l_d2 = layers.Dropout(0.2)(max_p2)
l3 = layers.BatchNormalization()(l_d2)
lstm = layers.LSTM(units = 64)(l3)
l4 = layers.Dense(128, activation = 'relu')(lstm)
output_layer = layers.Dense(num_classes, activation = 'sigmoid')(l4)

model_2 = tf.keras.Model(inputs = [text_input], outputs = output_layer)

In [18]:
model_2.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy']
)
model_2.summary()
hist2 = model_2.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 30, batch_size = 20, verbose = 2)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
from matplotlib.pyplot import plot
history_df = pd.DataFrame(hist2.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-entropy")
history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")